In [ ]:
import { StructuredOutputParser } from "@langchain/core/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  modelName: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const parser = StructuredOutputParser.fromNamesAndDescriptions({
  answer: "用户问题的答案，格式是以英文逗号隔开的中文字符串",
  evidence: "你回答用户问题所依据的答案",
  confidence: "问题答案的可信度评分，格式是百分数",
});
// parser.getFormatInstructions();

const prompt = PromptTemplate.fromTemplate(`
    列出3个 {country} 的著名的互联网公司.
    {instructions}
`);
const chain = prompt.pipe(llm).pipe(parser);
await chain.invoke({
  instructions: parser.getFormatInstructions(),
  country: "中国",
});


In [ ]:
import { CommaSeparatedListOutputParser } from "@langchain/core/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  modelName: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const parser = new CommaSeparatedListOutputParser();
// parser.getFormatInstructions();

const prompt = PromptTemplate.fromTemplate(`
    列出 {country} 排名前20的互联网公司.
    {instructions}
`);
const chain = prompt.pipe(llm).pipe(parser);
await chain.invoke({
  instructions: parser.getFormatInstructions(),
  country: "中国",
});


In [ ]:
import {
  StructuredOutputParser,
  OutputFixingParser,
} from "langchain/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { z } from "zod";

const llm = new ChatOpenAI({
  modelName: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const schema = z.object({
  answer: z.string().describe("用户问题的答案，以英文逗号隔开的3个中文字符串"),
  confidence: z
    .number()
    .min(0)
    .max(100)
    .describe("问题答案的可信度评分，满分 100"),
});

const parser = StructuredOutputParser.fromZodSchema(schema);

// const prompt = PromptTemplate.fromTemplate(`
//     列出3个 {country} 的著名的互联网公司.
//     {instructions}
// `);
// const chain = prompt.pipe(llm).pipe(parser);
// await chain.invoke({
//   instructions: parser.getFormatInstructions(),
//   country: "中国",
// });

const wrongOutput = {
  abc: ["阿里巴巴", "腾讯", "百度", "字节"],
  def: "90%",
};
const fixParser = OutputFixingParser.fromLLM(llm, parser);
await fixParser.parse(JSON.stringify(wrongOutput));


In [ ]:
// import { TextLoader } from "langchain/document_loaders/fs/text";
import { JSONLoader } from "langchain/document_loaders/fs/json";

// const loader = new TextLoader('documents/data.txt')
// const docs = await loader.load()
// console.log(docs)

const jsonLoader = new JSONLoader("documents/data.json", "/title");
// const jsonLoader = new JSONLinesLoader('data.jsonl', '/html')
const json = await jsonLoader.load();
console.log(json);


In [ ]:
import { PDFLoader } from "@langchain/community/document_loaders/fs/pdf";
import "pdf-parse";

const loader = new PDFLoader("documents/data.pdf");
const docs = await loader.load();
console.log(docs);


In [ ]:
import { DirectoryLoader } from "langchain/document_loaders/fs/directory";
import { TextLoader } from "langchain/document_loaders/fs/text";
import { PDFLoader } from "@langchain/community/document_loaders/fs/pdf";
import "pdf-parse";

const loader = new DirectoryLoader("./documents", {
  ".txt": (path) => new TextLoader(path),
  ".pdf": (path) => new PDFLoader(path, { splitPages: false }),
});
const docs = await loader.load();

console.log(docs);


In [ ]:
import { DocxLoader } from "@langchain/community/document_loaders/fs/docx";
import "mammoth";

const loader = new DocxLoader("documents/data.docx");

const docs = await loader.load();

console.log(docs);


In [ ]:
import { PPTXLoader } from "@langchain/community/document_loaders/fs/pptx";
import "officeparser";

const loader = new PPTXLoader("documents/data.pptx");

const docs = await loader.load();

console.log(docs);


In [ ]:
import { GithubRepoLoader } from "@langchain/community/document_loaders/web/github";
import "ignore";

const loader = new GithubRepoLoader(
  "https://github.com/Issho-lin/webrtc-demo",
  {
    branch: "main",
    recursive: false,
    unknown: "warn",
    ignorePaths: ["*.md", "*.json"],
  }
);
const docs = await loader.load();

console.log(docs);


In [ ]:
import { CheerioWebBaseLoader } from "@langchain/community/document_loaders/web/cheerio";
import "cheerio";

const loader = new CheerioWebBaseLoader(
  "https://kaiyi.cool/blog/github-copilot",
  {
    selector: "h3",
  }
);

const docs = await loader.load();

console.log(docs[0].pageContent);


In [ ]:
import { SerpAPILoader } from "@langchain/community/document_loaders/web/serpapi";

const apiKey =
  "185032b32dc536d633129d221f7f7be48629f0e4569138241d817449166e21ca";
const question = "Coffee";
const loader = new SerpAPILoader({ q: question, apiKey });
const docs = await loader.load();

console.log(docs);


In [ ]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { TextLoader } from "langchain/document_loaders/fs/text";

const loader = new TextLoader("documents/data.txt");
const docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 100,
  chunkOverlap: 20,
});

const splitDocs = await splitter.splitDocuments(docs);

console.log(splitDocs);

In [ ]:
import {
  SupportedTextSplitterLanguages,
  RecursiveCharacterTextSplitter,
} from "langchain/text_splitter";

console.log(SupportedTextSplitterLanguages);

const js = `
function myFunction(name,job){
	console.log("Welcome " + name + ", the " + job);
}

myFunction('Harry Potter','Wizard')

function forFunction(){
	for (let i=0; i<5; i++){
        console.log("这个数字是" + i)
	}
}

forFunction()
`;

const splitter = RecursiveCharacterTextSplitter.fromLanguage("js", {
  chunkSize: 64,
  chunkOverlap: 0,
});

await splitter.createDocuments([js]);


In [ ]:
import { TokenTextSplitter } from "langchain/text_splitter";

const text = "I stand before you today the representative of a family in grief, in a country in mourning before a world in shock.";

const splitter = new TokenTextSplitter({
  chunkSize: 10,
  chunkOverlap: 0,
});

await splitter.createDocuments([text]);